In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import warnings # current version of seaborn generates a bunch of warnings that we'll ignore
warnings.filterwarnings("ignore")
import seaborn as sns
sns.set(style="white", color_codes=True)
import matplotlib.pyplot as plt
plt.style.use("ggplot")
%matplotlib
from sklearn import linear_model
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.cross_validation import KFold
from scipy.stats import skew, boxcox
from sklearn.preprocessing import StandardScaler
import itertools
from datetime import datetime
import time
pd.set_option("display.max_columns",1000)
pd.set_option('display.max_rows',1000)

Using matplotlib backend: Qt5Agg


In [ ]:
1.027

# My functions

In [2]:
def logregobj(preds, dtrain):
    labels = dtrain.get_label()
    con = 2
    x = preds - labels
    grad = con * x / (np.abs(x) + con)
    hess = con ** 2 / (np.abs(x) + con) ** 2
    return grad, hess


def xg_eval_mae(yhat, dtrain):
    y = dtrain.get_label()
    return 'mae', mean_absolute_error(np.exp(y),
                                      np.exp(yhat))

In [3]:
def evalerror(preds, dtrain):
    labels = dtrain.get_label()
    return 'mae', mean_absolute_error(np.exp(preds), np.exp(labels))
def dif_feat(data,feat):
    new_data = pd.DateFrame()
    for i in range(len(feat)):
        for j in range(i+1,len(feat)):
            new_data[feat[i]+"-"+feat[j]] = data[feat[i]] - data[feat[j]]
        print(feat[i])
    return new_data
    del new_data
def transf_cat(train,test,feat_cat,feat_cont,fold):
    test['Proba'] = 0
    train['Proba'] = 0
    n_folds = fold
    kf = KFold(train.shape[0],n_folds)#,shuffle=True
    for train_index, val_index in kf:
        print("new_fold")
        training_data, validation_data = train.loc[train_index], train.loc[val_index]
        training_data['new_feat'] = training_data.groupby(feat_cat)[feat_cont].transform("mean")
        dat = training_data.groupby(feat_cat,as_index=False).mean()
        d={}
        for i in range(len(dat)):
            d[dat[feat_cat][i]] = dat['new_feat'][i]
        m=np.mean(training_data[feat_cont])
        def gr(x):
            try:
                return d[x]
            except:
                return m  
        train['Proba'].loc[val_index] = train[feat_cat].loc[val_index].apply(gr)
        test_new = test[feat_cat].apply(gr)
        test['Proba'] += np.array(test_new)
    return train['Proba'],test['Proba']*(1/n_folds)

In [ ]:
def get_prob_xgb(train,test,feat,fold,y,method,i):
    h = []
    proba = 0
    params = {
        'max_depth': 4,
        'eta': 0.1,
        'nthread': 4,
        'subsample': 0.8,
        'eval_metric': ['mae'],
        'objective': 'reg:linear',
        'colsample_bytree': 0.6,
        'min_child_weight': 0,
        #'scale_pos_weight':(1-train.Label.mean())/train.Label.mean(),
        'seed':7
    }
    train['pred'+method+str(i)]=0
    test['pred'+method+str(i)] =0 
    kf = KFold(train.shape[0], n_folds=fold,shuffle=True)
    for train_index, val_index in kf:
        training_data, validation_data = train.loc[train_index], train.loc[val_index]
        dtr = xgb.DMatrix(training_data[feat], label=training_data[y])
        dval = xgb.DMatrix(validation_data[feat], label=validation_data[y])
        watchlist = [(dtr, 'train'), (dval, 'eval')]
        history = dict()
        model0 = xgb.train(params, dtr, num_boost_round=1000, evals=watchlist,evals_result=history, verbose_eval=10)
        train['pred'+method].loc[val_index] = model0.predict_proba(dval)
        dtest = xgb.DMatrix(test[feat])
        proba = proba+model0.predict(dtest)
        imps = pd.Series(model0.get_fscore()).sort_values()
        h.append(imps)
    return proba*(1/fold)

In [ ]:
shift = 200
def get_prob_lin(train,test,feat,fold,y,method,i):
    model0 = linear_model.LinearRegression()
    train['pred'+method+str(i)]=0
    test['pred'+method+str(i)] =0 
    er = []
    proba = 0
    kf = KFold(train.shape[0], n_folds=fold,shuffle=True)
    for train_index, val_index in kf:
        training_data, validation_data = train.loc[train_index], train.loc[val_index]
        model0.fit(np.log(training_data[feat]+shift),np.log(training_data[y]+shift))
        pred = model0.predict(np.log(validation_data[feat]+shift))
        pred = np.exp(pred)-shift
        er.append(mean_absolute_error(pred,validation_data[y]))
        print("mae",mean_absolute_error(pred,validation_data[y]))
        train['pred'+method+str(i)].loc[val_index] = pred
        proba = proba+model0.predict(np.log(test[feat]+shift))
    print("mean",np.mean(er))
    test['pred'+method+str(i)]=(np.exp(proba)-shift)*(1/fold)

# TRAIN TEST

In [4]:
train = pd.read_csv("data/train.csv")
train.head()

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,cat11,cat12,cat13,cat14,cat15,cat16,cat17,cat18,cat19,cat20,cat21,cat22,cat23,cat24,cat25,cat26,cat27,cat28,cat29,cat30,cat31,cat32,cat33,cat34,cat35,cat36,cat37,cat38,cat39,cat40,cat41,cat42,cat43,cat44,cat45,cat46,cat47,cat48,cat49,cat50,cat51,cat52,cat53,cat54,cat55,cat56,cat57,cat58,cat59,cat60,cat61,cat62,cat63,cat64,cat65,cat66,cat67,cat68,cat69,cat70,cat71,cat72,cat73,cat74,cat75,cat76,cat77,cat78,cat79,cat80,cat81,cat82,cat83,cat84,cat85,cat86,cat87,cat88,cat89,cat90,cat91,cat92,cat93,cat94,cat95,cat96,cat97,cat98,cat99,cat100,cat101,cat102,cat103,cat104,cat105,cat106,cat107,cat108,cat109,cat110,cat111,cat112,cat113,cat114,cat115,cat116,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss
0,1,A,B,A,B,A,A,A,A,B,A,B,A,A,A,A,A,A,A,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A,D,B,B,D,D,B,D,C,B,D,B,A,A,A,A,A,D,B,C,E,A,C,T,B,G,A,A,I,E,G,J,G,BU,BC,C,AS,S,A,O,LB,0.726300,0.245921,0.187583,0.789639,0.310061,0.718367,0.335060,0.30260,0.67135,0.83510,0.569745,0.594646,0.822493,0.714843,2213.18
1,2,A,B,A,A,A,A,A,A,B,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,D,B,B,D,D,A,B,C,B,D,B,A,A,A,A,A,D,D,C,E,E,D,T,L,F,A,A,E,E,I,K,K,BI,CQ,A,AV,BM,A,O,DP,0.330514,0.737068,0.592681,0.614134,0.885834,0.438917,0.436585,0.60087,0.35127,0.43919,0.338312,0.366307,0.611431,0.304496,1283.60
2,5,A,B,A,A,B,A,A,A,B,B,B,B,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,D,B,B,B,D,B,D,C,B,B,B,A,A,A,A,A,D,D,C,E,E,A,D,L,O,A,B,E,F,H,F,A,AB,DK,A,C,AF,A,I,GK,0.261841,0.358319,0.484196,0.236924,0.397069,0.289648,0.315545,0.27320,0.26076,0.32446,0.381398,0.373424,0.195709,0.774425,3005.09
3,10,B,B,A,B,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A,A,A,D,B,B,D,D,D,B,C,B,D,B,A,A,A,A,A,D,D,C,E,E,D,T,I,D,A,A,E,E,I,K,K,BI,CS,C,N,AE,A,O,DJ,0.321594,0.555782,0.527991,0.373816,0.422268,0.440945,0.391128,0.31796,0.32128,0.44467,0.327915,0.321570,0.605077,0.602642,939.85
4,11,A,B,A,B,A,A,A,A,B,B,A,B,A,A,A,A,A,A,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A,A,A,A,D,B,D,B,D,B,B,C,B,B,C,A,A,A,B,H,D,B,D,E,E,A,P,F,J,A,A,D,E,K,G,B,H,C,C,Y,BM,A,K,CK,0.273204,0.159990,0.527991,0.473202,0.704268,0.178193,0.247408,0.24564,0.22089,0.21230,0.204687,0.202213,0.246011,0.432606,2763.85


In [5]:
test = pd.read_csv("data/test.csv")
test.head()

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,cat11,cat12,cat13,cat14,cat15,cat16,cat17,cat18,cat19,cat20,cat21,cat22,cat23,cat24,cat25,cat26,cat27,cat28,cat29,cat30,cat31,cat32,cat33,cat34,cat35,cat36,cat37,cat38,cat39,cat40,cat41,cat42,cat43,cat44,cat45,cat46,cat47,cat48,cat49,cat50,cat51,cat52,cat53,cat54,cat55,cat56,cat57,cat58,cat59,cat60,cat61,cat62,cat63,cat64,cat65,cat66,cat67,cat68,cat69,cat70,cat71,cat72,cat73,cat74,cat75,cat76,cat77,cat78,cat79,cat80,cat81,cat82,cat83,cat84,cat85,cat86,cat87,cat88,cat89,cat90,cat91,cat92,cat93,cat94,cat95,cat96,cat97,cat98,cat99,cat100,cat101,cat102,cat103,cat104,cat105,cat106,cat107,cat108,cat109,cat110,cat111,cat112,cat113,cat114,cat115,cat116,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14
0,4,A,B,A,A,A,A,A,A,B,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,D,B,B,D,D,B,B,C,B,D,B,A,A,A,A,A,D,C,C,E,C,D,T,H,G,A,A,G,E,I,L,K,BI,BC,A,J,AX,A,Q,HG,0.321594,0.299102,0.246911,0.402922,0.281143,0.466591,0.317681,0.61229,0.34365,0.38016,0.377724,0.369858,0.704052,0.392562
1,6,A,B,A,B,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,B,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A,A,B,A,D,B,B,D,D,B,B,C,B,B,B,A,A,A,A,A,D,D,D,E,A,A,P,B,D,A,A,G,G,G,F,B,BI,CO,E,G,X,A,L,HK,0.634734,0.620805,0.654310,0.946616,0.836443,0.482425,0.443760,0.71330,0.51890,0.60401,0.689039,0.675759,0.453468,0.208045
2,9,A,B,A,B,B,A,B,A,B,B,A,B,B,B,A,A,A,A,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,B,A,B,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,B,D,B,B,B,B,B,D,C,B,B,B,A,B,A,A,A,D,D,C,E,E,A,D,G,Q,A,D,D,E,J,G,A,BI,CS,C,U,AE,A,K,CK,0.290813,0.737068,0.711159,0.412789,0.718531,0.212308,0.325779,0.29758,0.34365,0.30529,0.245410,0.241676,0.258586,0.297232
3,12,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A,B,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A,A,A,A,D,B,D,B,D,B,B,A,B,D,D,A,A,A,G,H,D,D,C,E,E,D,T,G,A,A,D,E,E,I,K,K,BI,CR,A,AY,AJ,A,P,DJ,0.268622,0.681761,0.592681,0.354893,0.397069,0.369930,0.342355,0.40028,0.33237,0.31480,0.348867,0.341872,0.592264,0.555955
4,15,B,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,D,B,B,D,D,B,B,C,B,B,B,A,A,A,A,A,D,B,D,E,A,A,P,A,A,A,A,F,E,G,E,B,AB,EG,A,E,I,C,J,HA,0.553846,0.299102,0.263570,0.696873,0.302678,0.398862,0.391833,0.23688,0.43731,0.50556,0.359572,0.352251,0.301535,0.825823


# feat

In [6]:
feat_base = train.columns
cat = feat_base[1:117]
cont = feat_base[117:-1]
y = train.loss

In [ ]:
sub['c'] = 0

In [ ]:
tr = pd.DataFrame()
tr['id'] = train['id']
tr['loss'] = train['loss']
tr['c'] = 1
tr.head()

In [ ]:
d = pd.concat([tr,sub])
d.head()

In [ ]:
plt.scatter(tr.id,tr.loss,c='r', s=10, marker='o', alpha=.4)
plt.scatter(sub.id,sub.loss,c='b', s=10, marker='o', alpha=.4)
plt.show()

In [ ]:
plt.scatter(d.id,d.loss,c=d.c, s=10, marker='o', alpha=.1)
plt.show()

In [ ]:
sub.head()

# Vizualization and description

In [ ]:
desc = train.describe()
desc

In [ ]:
test.describe()

In [ ]:
train.describe(include=[object])

In [ ]:
test.describe(include=[object])

In [ ]:
plt.scatter(train.id,y,c=train.c, s=10, marker='o', alpha=.1)
plt.show()

In [ ]:
plt.scatter(train.cont1,train.cont2,c=train.c, s=5, marker='o', alpha=0.2)
plt.show()

In [ ]:
plt.scatter(train.id,np.log(y+1),color='r', s=10, marker='o', alpha=.99)

In [ ]:
np.log(y[y]+200).hist(bins=30)


In [ ]:
razmah = desc['loss']['75%']+1.5*(desc['loss']['75%']-desc['loss']['25%'])+20000
razmah

In [ ]:
def get_loss(x):
    if x<razmah:
        return 0
    else:
        return 1
train['c'] = train['loss'].apply(get_loss)

In [ ]:
#sns.clustermap(train)

In [ ]:
sns.jointplot(x="cont1", y="cont3", data=train, size=5)
plt.show()

In [ ]:
len(train[train.loss>razmah])

In [ ]:
for i in range(0,len(cat)):
    sns.boxplot(x=cat[i], y="loss", data=train)
    #ax = sns.boxplot(x=cat[i], y="loss", data=train)
    #ax = sns.stripplot(x=cat[i], y="loss", data=train, jitter=False, edgecolor="gray")
    plt.show()

In [ ]:
for i in range(0,len(cont)):
    for j in range(i+1,len(cont)):
        sns.FacetGrid(train, hue="c", size=5).map(plt.scatter, cont[i], cont[j]).add_legend()
        plt.show()

In [ ]:
sns.lmplot('cont1', 'cont2',data=train, 
           fit_reg=False,
           hue="c",  
           scatter_kws={"marker": "*", 
                        "s": 3})
plt.show()

In [ ]:
sns.violinplot(x=cat[0], y="loss", data=train, size=6)
plt.show()

In [ ]:
x=cat[i], y="loss", data=train

In [ ]:
sns.FacetGrid(train, hue="c", size=6).map(sns.kdeplot, "cont13").add_legend()
plt.show()

In [ ]:
sns.pairplot(train[cont], hue=train.c, size=3)
#sns.pairplot(iris.drop("Id", axis=1), hue="Species", size=3, diag_kind="kde")
plt.show()

In [ ]:
z = list(cont)

In [ ]:
z.extend('c')
z

In [ ]:
train[z].boxplot(by='c', figsize=(12, 6))
plt.show()

In [ ]:
# One cool more sophisticated technique pandas has available is called Andrews Curves
# Andrews Curves involve using attributes of samples as coefficients for Fourier series
# and then plotting these
from pandas.tools.plotting import andrews_curves
andrews_curves(train[z], "c")

In [ ]:
# Another multivariate visualization technique pandas has is parallel_coordinates
# Parallel coordinates plots each feature on a separate column & then draws lines
# connecting the features for each data sample
from pandas.tools.plotting import parallel_coordinates
parallel_coordinates(train[z], "c")

In [ ]:
# A final multivariate visualization technique pandas has is radviz
# Which puts each feature as a point on a 2D plane, and then simulates
# having each sample attached to those points through a spring weighted
# by the relative value for that feature
from pandas.tools.plotting import radviz
radviz(iris.drop("Id", axis=1), "Species")

In [ ]:
REGR:1951
XGB:datat
RegrLog:1307
    
factor:train-mae:1101.17	eval-mae:1180.22
get_dum 1177

# Clastering

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
y_pred = KMeans(n_clusters=2, random_state= 6).fit_predict(train[cont])

In [ ]:
y_pred

In [ ]:
plt.scatter(train['cont1'], train['loss'], c=y_pred)
plt.show()

In [ ]:
for i in range(0,len(cont)):
    plt.scatter(train[cont[i]], train['loss'], c=y_pred)
    plt.show()

In [ ]:
from sklearn.cluster import MeanShift, estimate_bandwidth

In [ ]:
# The following bandwidth can be automatically detected using
#bandwidth = estimate_bandwidth(train[cont], quantile=0.2, n_samples=10)
#ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
ms = MeanShift( bin_seeding=True)
ms.fit(train[cont])
labels = ms.labels_
cluster_centers = ms.cluster_centers_

labels_unique = np.unique(labels)
n_clusters_ = len(labels_unique)

print("number of estimated clusters : %d" % n_clusters_)

In [ ]:
import matplotlib.pyplot as plt
from itertools import cycle

plt.figure(1)
plt.clf()

colors = cycle('bgrcmykbgrcmykbgrcmykbgrcmyk')
for k, col in zip(range(n_clusters_), colors):
    my_members = labels == k
    cluster_center = cluster_centers[k]
    plt.plot(X[my_members, 0], X[my_members, 1], col + '.')
    plt.plot(cluster_center[0], cluster_center[1], 'o', markerfacecolor=col,
             markeredgecolor='k', markersize=14)
plt.title('Estimated number of clusters: %d' % n_clusters_)
plt.show()

# NewData

In [7]:
test['loss'] = np.nan

In [8]:
data = pd.concat([train,test],ignore_index=True)
data.head()

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,cat11,cat12,cat13,cat14,cat15,cat16,cat17,cat18,cat19,cat20,cat21,cat22,cat23,cat24,cat25,cat26,cat27,cat28,cat29,cat30,cat31,cat32,cat33,cat34,cat35,cat36,cat37,cat38,cat39,cat40,cat41,cat42,cat43,cat44,cat45,cat46,cat47,cat48,cat49,cat50,cat51,cat52,cat53,cat54,cat55,cat56,cat57,cat58,cat59,cat60,cat61,cat62,cat63,cat64,cat65,cat66,cat67,cat68,cat69,cat70,cat71,cat72,cat73,cat74,cat75,cat76,cat77,cat78,cat79,cat80,cat81,cat82,cat83,cat84,cat85,cat86,cat87,cat88,cat89,cat90,cat91,cat92,cat93,cat94,cat95,cat96,cat97,cat98,cat99,cat100,cat101,cat102,cat103,cat104,cat105,cat106,cat107,cat108,cat109,cat110,cat111,cat112,cat113,cat114,cat115,cat116,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss
0,1,A,B,A,B,A,A,A,A,B,A,B,A,A,A,A,A,A,A,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A,D,B,B,D,D,B,D,C,B,D,B,A,A,A,A,A,D,B,C,E,A,C,T,B,G,A,A,I,E,G,J,G,BU,BC,C,AS,S,A,O,LB,0.726300,0.245921,0.187583,0.789639,0.310061,0.718367,0.335060,0.30260,0.67135,0.83510,0.569745,0.594646,0.822493,0.714843,2213.18
1,2,A,B,A,A,A,A,A,A,B,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,D,B,B,D,D,A,B,C,B,D,B,A,A,A,A,A,D,D,C,E,E,D,T,L,F,A,A,E,E,I,K,K,BI,CQ,A,AV,BM,A,O,DP,0.330514,0.737068,0.592681,0.614134,0.885834,0.438917,0.436585,0.60087,0.35127,0.43919,0.338312,0.366307,0.611431,0.304496,1283.60
2,5,A,B,A,A,B,A,A,A,B,B,B,B,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,D,B,B,B,D,B,D,C,B,B,B,A,A,A,A,A,D,D,C,E,E,A,D,L,O,A,B,E,F,H,F,A,AB,DK,A,C,AF,A,I,GK,0.261841,0.358319,0.484196,0.236924,0.397069,0.289648,0.315545,0.27320,0.26076,0.32446,0.381398,0.373424,0.195709,0.774425,3005.09
3,10,B,B,A,B,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A,A,A,D,B,B,D,D,D,B,C,B,D,B,A,A,A,A,A,D,D,C,E,E,D,T,I,D,A,A,E,E,I,K,K,BI,CS,C,N,AE,A,O,DJ,0.321594,0.555782,0.527991,0.373816,0.422268,0.440945,0.391128,0.31796,0.32128,0.44467,0.327915,0.321570,0.605077,0.602642,939.85
4,11,A,B,A,B,A,A,A,A,B,B,A,B,A,A,A,A,A,A,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A,A,A,A,D,B,D,B,D,B,B,C,B,B,C,A,A,A,B,H,D,B,D,E,E,A,P,F,J,A,A,D,E,K,G,B,H,C,C,Y,BM,A,K,CK,0.273204,0.159990,0.527991,0.473202,0.704268,0.178193,0.247408,0.24564,0.22089,0.21230,0.204687,0.202213,0.246011,0.432606,2763.85


# KAGGLE MAGIC

In [ ]:
COMB_FEATURE = 'cat80,cat87,cat57,cat12,cat79,cat10,cat7,cat89,cat2,cat72,' \
               'cat81,cat11,cat1,cat13,cat9,cat3,cat16,cat90,cat23,cat36,' \
               'cat73,cat103,cat40,cat28,cat111,cat6,cat76,cat50,cat5,' \
               'cat4,cat14,cat38,cat24,cat82,cat25'.split(',')

def encode(charcode):
    r = 0
    ln = len(str(charcode))
    for i in range(ln):
        r += (ord(str(charcode)[i]) - ord('A') + 1) * 26 ** (ln - i - 1)
    return r

fair_constant = 2
def fair_obj(preds, dtrain):
    labels = dtrain.get_label()
    x = (preds - labels)
    den = abs(x) + fair_constant
    grad = fair_constant * x / (den)
    hess = fair_constant * fair_constant / (den * den)
    return grad, hess

def xg_eval_mae(yhat, dtrain):
    y = dtrain.get_label()
    return 'mae', mean_absolute_error(np.exp(y)-shift,
                                      np.exp(yhat)-shift)
def mungeskewed(train, test, numeric_feats):
    ntrain = train.shape[0]
    test['loss'] = 0
    train_test = pd.concat((train, test)).reset_index(drop=True)
    skewed_feats = train[numeric_feats].apply(lambda x: skew(x.dropna()))
    skewed_feats = skewed_feats[skewed_feats > 0.25]
    skewed_feats = skewed_feats.index

    for feats in skewed_feats:
        train_test[feats] = train_test[feats] + 1
        train_test[feats], lam = boxcox(train_test[feats])
    return train_test, ntrain

In [ ]:
boxcox(train['cont14']+1)

In [ ]:
numeric_feats = [x for x in train.columns[1:-1] if 'cont' in x]
categorical_feats = [x for x in train.columns[1:-1] if 'cat' in x]
train_test, ntrain = mungeskewed(train, test, numeric_feats)

In [ ]:
train_test.head()

In [ ]:
for comb in itertools.combinations(COMB_FEATURE, 2):
        feat = comb[0] + "_" + comb[1]
        train_test[feat] = train_test[comb[0]] + train_test[comb[1]]
        train_test[feat] = train_test[feat].apply(encode)
        print('Analyzing Columns:', feat)

In [ ]:
categorical_feats = [x for x in train_test.columns[1:] if 'cat' in x]

In [ ]:
for col in categorical_feats:
        print('Analyzing Column:', col)
        train_test[col] = train_test[col].apply(encode)

In [ ]:
ss = StandardScaler()
train_test[numeric_feats] = \
        ss.fit_transform(train_test[numeric_feats].values)

In [ ]:
train = train_test.iloc[:ntrain, :].copy()
test = train_test.iloc[ntrain:, :].copy()

# GET DOMMIES

In [ ]:
for column in list(train.select_dtypes(include=['object']).columns):
    if train[column].nunique() != test[column].nunique():
        set_train = set(train[column].unique())
        set_test = set(test[column].unique())
        remove_train = set_train - set_test
        remove_test = set_test - set_train

        remove = remove_train.union(remove_test)
        def filter_cat(x):
            if x in remove:
                return np.nan
            return x

        data[column] = data[column].apply(lambda x: filter_cat(x), 1)
            
    data[column] = pd.factorize(data[column].values, sort=True)[0]

In [ ]:
cat = data.columns[1:-15]
cat[-1],cat[0]

In [ ]:
data=pd.get_dummies(data,columns = cat).rename(columns=lambda x: str(x))
print(data.shape)
data.head()

In [ ]:
data.columns[16:]

In [ ]:
np.int8(1)

In [ ]:
for i in data.columns[16:]:
    data[i] = data[i].astype(np.int8)

In [ ]:
data.info()

In [ ]:
data.to_csv("datap/data_get_dum_col_nan_bool.csv")

In [ ]:
train = data[data.Test == 0].drop('Test', axis=1).copy()
test = data[data.Test == 1].drop('Test', axis=1).copy()

In [ ]:
for i in train.columns:
    if mean(train[i])==0:
        train.drop([i],axis=1,inplace=True)

In [ ]:
for i in test.columns:
    if mean(test[i])==0:
        test.drop([i],axis=1,inplace=True)

In [ ]:
test.columns[15:]

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
train.columns[16:]

# del cat 0 cat-1

In [ ]:
data.head()

In [ ]:
data.drop(['cat{}_0'.format(i)],axis=1,inplace=True)

In [ ]:
for i in range(0,117):
    try:
        data.drop(['cat{}_0'.format(i)],axis=1,inplace=True)
    except:
        print(i)

In [ ]:
cat113_0

# Count factorize

In [ ]:
for column in list(train.select_dtypes(include=['object']).columns):
    data[column] = data.groupby(column).transform('count')
data.head()

# feature factor

In [ ]:
data.head()

In [ ]:
for column in list(train.select_dtypes(include=['object']).columns):
    data[column] = data.groupby(column)['cont14'].transform("mean")
data.head()

# FACTORIZE

In [ ]:
for column in list(train.select_dtypes(include=['object']).columns):
    if train[column].nunique() != test[column].nunique():
        set_train = set(train[column].unique())
        set_test = set(test[column].unique())
        remove_train = set_train - set_test
        remove_test = set_test - set_train

        remove = remove_train.union(remove_test)
        def filter_cat(x):
            if x in remove:
                return np.nan
            return x

        data[column] = data[column].apply(lambda x: filter_cat(x), 1)
            
    data[column] = pd.factorize(data[column].values, sort=True)[0]


In [ ]:
data.head()

In [ ]:
for i in list(data.columns[1:-15]):
    data[i]= pd.factorize(data[i])[0]
data.head()

In [ ]:
data

In [ ]:
feat=list(set(train.columns)-{"id"})

# Group mean(fold)

In [ ]:
def transf_cat(train,test,feat_cat,feat_cont,fold):
    test['Proba'] = 0
    train['Proba'] = 0
    n_folds = fold
    kf = KFold(train.shape[0],n_folds)#,shuffle=True
    for train_index, val_index in kf:
        print("new_fold")
        training_data, validation_data = train.loc[train_index], train.loc[val_index]
        training_data['new_feat'] = training_data.groupby(feat_cat)[feat_cont].transform("mean")
        dat = training_data.groupby(feat_cat,as_index=False).mean()
        d={}
        for i in range(len(dat)):
            d[dat[feat_cat][i]] = dat['new_feat'][i]
        m=np.mean(training_data[feat_cont])
        def gr(x):
            try:
                return d[x]
            except:
                return m  
        train['Proba'].loc[val_index] = train[feat_cat].loc[val_index].apply(gr)
        test_new = test[feat_cat].apply(gr)
        test['Proba'] += np.array(test_new)
    return train['Proba'],test['Proba']*(1/n_folds)


In [ ]:
train = data[data['loss'].notnull()].reset_index()
test = data[data['loss'].isnull()].reset_index()
train.drop(['id','index'],axis=1,inplace=True)
test.drop(['id','index'],axis=1,inplace=True)

In [ ]:
cat = list(train.columns[:-15])

In [ ]:
for i in cat:
    print(i)
    %time train[i+"n"] , test[i+"n"] = transf_cat(train,test,i,"loss",5)
    print("next_cat")

In [ ]:
train.drop(cat,axis=1,inplace=True)
train.drop(['Proba'],axis=1,inplace=True)
test.drop(cat,axis=1,inplace=True)
test.drop(['Proba'],axis=1,inplace=True)

In [ ]:
train.head()


In [ ]:
test.head()

In [ ]:
train.to_csv('datap/train5foldmean.csv',index=False)
test.to_csv('datap/test5foldmean.csv',index=False)

# Group_kfold(median)

In [ ]:
def transf_cat(train,test,feat_cat,feat_cont,fold):
    test['Proba'] = 0
    train['Proba'] = 0
    n_folds = fold
    kf = KFold(train.shape[0],n_folds)#,shuffle=True
    for train_index, val_index in kf:
        print("new_fold")
        training_data, validation_data = train.loc[train_index], train.loc[val_index]
        training_data['new_feat'] = training_data.groupby(feat_cat)[feat_cont].transform("median")
        dat = training_data.groupby(feat_cat,as_index=False).median()
        d={}
        for i in range(len(dat)):
            d[dat[feat_cat][i]] = dat['new_feat'][i]
        m=np.median(training_data[feat_cont])
        def gr(x):
            try:
                return d[x]
            except:
                return m  
        train['Proba'].loc[val_index] = train[feat_cat].loc[val_index].apply(gr)
        test_new = test[feat_cat].apply(gr)
        test['Proba'] += np.array(test_new)
    return train['Proba'],test['Proba']*(1/n_folds)


In [ ]:
train = data[data['loss'].notnull()].reset_index()
test = data[data['loss'].isnull()].reset_index()
train.drop(['id','index'],axis=1,inplace=True)
test.drop(['id','index'],axis=1,inplace=True)

In [ ]:
cat = list(train.columns[:-15])
cat[-1],cat[0]

In [ ]:
for i in cat:
    print(i)
    %time train[i+"n"] , test[i+"n"] = transf_cat(train,test,i,"loss",5)
    print("next_cat")

In [ ]:
train.drop(cat,axis=1,inplace=True)
train.drop(['Proba'],axis=1,inplace=True)
test.drop(cat,axis=1,inplace=True)
test.drop(['Proba'],axis=1,inplace=True)

In [ ]:
train.to_csv('datap/trainkagglebox.csv',index=False)
test.to_csv('datap/testkagglebox.csv',index=False)

In [ ]:
#pd.read_csv('datap/train5fold.csv')

In [ ]:
с = х

# NewFeatches From Kaggle

In [ ]:
data["cont1"] = np.sqrt(preprocessing.minmax_scale(data["cont1"]))
data["cont4"] = np.sqrt(preprocessing.minmax_scale(data["cont4"]))
data["cont5"] = np.sqrt(preprocessing.minmax_scale(data["cont5"]))
data["cont8"] = np.sqrt(preprocessing.minmax_scale(data["cont8"]))
data["cont10"] = np.sqrt(preprocessing.minmax_scale(data["cont10"]))
data["cont11"] = np.sqrt(preprocessing.minmax_scale(data["cont11"]))
data["cont12"] = np.sqrt(preprocessing.minmax_scale(data["cont12"]))

data["cont6"] = np.log(preprocessing.minmax_scale(data["cont6"])+0000.1)
data["cont7"] = np.log(preprocessing.minmax_scale(data["cont7"])+0000.1)
data["cont9"] = np.log(preprocessing.minmax_scale(data["cont9"])+0000.1)
data["cont13"] = np.log(preprocessing.minmax_scale(data["cont13"])+0000.1)
data["cont14"]=(np.maximum(data["cont14"]-0.179722,0)/0.665122)**0.25

In [ ]:
train.head()

In [ ]:
train.shape

# VALIDATION

In [9]:
train = pd.read_csv("datap/train3foldmean.csv")
test = pd.read_csv("datap/test3foldmean.csv")

In [10]:
train["cont15"]=(np.maximum(train["cont14"]-0.179722,0))
test["cont15"]=(np.maximum(test["cont14"]-0.179722,0))

In [ ]:
data.drop(["id"],axis=1,inplace=True)

In [ ]:
data.head()

In [ ]:
data['id'] = list(d['id'])
data['id'] = data['id']*(1/len(data))

In [ ]:
train = data[data['loss'].notnull()]
test = data[data['loss'].isnull()]

In [69]:
Xtr, Xval, ytr, yval = train_test_split(train,np.array(train.loss),random_state = 7, test_size=0.2)


In [75]:
Xtr.drop(['loss'],axis=1,inplace=True)

In [70]:
shift = 200
feat = list(set(test.columns)-{"loss",'id'} )

In [110]:
len(train[train['loss']<1000].loss)/len(train)

0.18052974224450133

In [82]:
from sklearn.ensemble import RandomForestRegressor


In [106]:
rf = RandomForestRegressor(n_estimators=200, n_jobs=-1)
rf.fit(Xtr[feat], np.log(ytr+shift))
#log_loss(yval, rf.predict_proba(Xval))

KeyboardInterrupt: 

In [86]:
pred = rf.predict(Xval[feat])

In [103]:
len(pred),len(yval)

(37664, 37664)

In [105]:
mean_absolute_error(pred,yval)

1245.0135099604395

In [104]:
mean_absolute_error(np.exp(np.array(pred))-shift,yval)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [46]:
import os
from pylightgbm.models import GBMRegressor
#os.environ['LIGHTGBM_EXEC'] = "/root/tools/LightGBM/lightgbm"
os.environ['LIGHTGBM_EXEC'] = "/home/dz/anaconda3/LightGBM/lightgbm"

In [49]:
clf = GBMRegressor(
    #exec_path='/path/to/your/LightGBM/lightgbm',
    exec_path='/home/dz/anaconda3/LightGBM/lightgbm',
    num_threads=-1,
    learning_rate = 0.01,
    num_iterations = 5000, 
    verbose = False, 
    early_stopping_round = 5,
    feature_fraction = 0.75,
    bagging_fraction = 0.9,
) 

In [51]:
%%time
clf.fit(Xtr, np.log(ytr + shift))
y_pred_2 = np.exp(clf.predict(Xval)) - shift
print(mean_absolute_error(y_pred_2, yval))

PermissionError: [Errno 13] Permission denied

In [41]:
%%time
params = {'verbose': False, 'num_iterations': 100}
clf = GBMRegressor(**params)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 85.4 µs


In [ ]:
%%time
from pylightgbm.models import GBMRegressor
params = {'verbose': False, 'num_iterations': 100}
clf = GBMRegressor(**params)
clf.fit(train, np.log(y_train + SHIFT))
y_pred_2 = np.exp(clf.predict(test)) - SHIFT
print(mean_absolute_error(y_pred_2, y_test))

In [ ]:
feat = list(set(test.columns)-{"id","loss"} )

In [ ]:
feat = list(set(test.columns)-{"id","loss"} - set(cat))

In [ ]:
Xtr[feat].head()

In [ ]:
del train

# Linear_model on cont

In [ ]:
Xtr[feat].shape

In [ ]:
regr = linear_model.LinearRegression()

In [ ]:

regr.fit(np.log(Xtr[feat]+shift),np.log(ytr+shift))

In [ ]:
#regr.coef_

In [ ]:
pred = regr.predict(np.log(Xval[feat]+shift))

In [ ]:
MAE = sum(abs(yval-(np.exp(pred)-shift))) / len(yval)
MAE

In [ ]:
mean_absolute_error(np.exp(pred)-shift,yval)

In [ ]:
gal(1114 fact 1274)
1258 med 5

# XGBOOST

In [58]:
dtr = xgb.DMatrix(Xtr[feat], label=np.log(ytr+shift))
dval = xgb.DMatrix(Xval[feat], label=np.log(yval+shift))
watchlist = [(dtr, 'train'), (dval, 'eval')]
history = dict()

In [ ]:
del Xtr,Xval

In [ ]:
params = {
    'max_depth': 4,
    'eta': 0.1,
    'nthread': 4,
    'subsample': 0.8,
    'eval_metric': ['mae'],
    'objective': 'reg:linear',
    'colsample_bytree': 0.6,
    'min_child_weight': 0,
    #'scale_pos_weight':(1-train.Label.mean())/train.Label.mean(),
    'seed':7
}

In [ ]:
plt.plot([1000, 1001, 1002])  # scales with m = 1, c = -1000
plt.show()

In [ ]:
params = {
            'min_child_weight': 1,
            'eta': 0.1,
            'colsample_bytree': 0.5,
            'max_depth': 12,
            'subsample': 0.8,
            'alpha': 1,
            'gamma': 1,
            'silent': 1,
            'verbose_eval': True,
            'seed':7
    }

In [ ]:
params={
    'seed': 1,
    'colsample_bytree': 0.3085,
    'subsample': 0.9930,
    'eta': 0.1,
    'gamma': 0.5290,
    'booster' :  'gbtree',    
    'objective': 'reg:linear',
    'max_depth': 7,
    'min_child_weight': 4.2922,
    'eval_metric': 'mae'
}

In [ ]:
RANDOM_STATE = 2016
params = {
        'min_child_weight': 1,
        'eta': 0.01,
        'colsample_bytree': 0.5,
        'max_depth': 12,
        'subsample': 0.8,
        'alpha': 1,
        'gamma': 1,
        'silent': 1,
        'verbose_eval': True,
        'seed': RANDOM_STATE
    }

In [ ]:
xgb_params = {
        'seed': 7,
        'gamma': 0.9,
        'alpha': 1,
        'colsample_bytree': 0.7,
        'silent': 0,
        'subsample': 0.7,
        'learning_rate': 0.01,
        'objective': 'reg:linear',
        'verbose_eval': True,
        'max_depth': 12,
        'min_child_weight': 1,
        'booster': 'gbtree',
    }

In [ ]:
params = { 'min_child_weight': 1.3921626196385004, 'eta': 0.00569207582224922, 'colsample_bytree': 0.6693550027860175, 'max_depth': 12, 'subsample': 0.6980320503253816, 'alpha': 0.9925559487305438, 'gamma': 0.904325256736723, 'silent': 0, 'verbose_eval': True, 'seed': RANDOM_STATE }

In [ ]:
xgb.cv(params, dtr, 2000, nfold=5,
       metrics={'mae'}, seed = 7,verbose_eval=50,
       callbacks=[xgb.callback.print_evaluation(show_stdv=True)])

In [ ]:
model = xgb.train(params, dtr, num_boost_round=2000, evals=watchlist,evals_result=history, verbose_eval=50)

In [ ]:
pred_xgb = model.predict(dval)

In [ ]:
mean_absolute_error(np.exp(pred_xgb)-shift,yval)

In [ ]:
dtest = xgb.DMatrix(test[feat])

In [ ]:
pred_xgb = model.predict(dtest)

# pred kaggle

In [ ]:
1133
1134 med 5

In [ ]:
def evalerror(preds, dtrain):
    labels = dtrain.get_label()
    return 'mae', mean_absolute_error(np.exp(preds), np.exp(labels))

In [ ]:
model = xgb.train(params, dtr,num_boost_round= int(2012 / 0.9), evals=watchlist,evals_result=history,  feval=evalerror,verbose_eval=50)

In [ ]:
count model 1131

In [ ]:
xgtrain = xgb.DMatrix(train[feat], label=np.log(train.loss+shift))
xgtest = xgb.DMatrix(test[feat])



In [ ]:
model = xgb.train(params, xgtrain, int(2012 / 0.9), feval=evalerror)

predicti = np.exp(model.predict(xgtest)) - shift

# Stacking

In [ ]:
feat_all = list(set(train.columns) -{'id','loss'})
feat_all

In [ ]:
shift=200
def get_prob_xgb(train,test,feat,fold,y,method,i):
    h = []
    er = []
    proba = 0
    params = {
            'min_child_weight': 10,
            'eta': 0.025,
            'colsample_bytree': 0.7,
            'max_depth': 10,
            'subsample': 0.9,
            'alpha': 1,
            'gamma': 1,
            'silent': 1,
            'verbose_eval': True,
            'seed': 7
        }
    train['pred'+method+str(i)]=0
    test['pred'+method+str(i)] =0 
    kf = KFold(train.shape[0], n_folds=fold,shuffle=True)
    for train_index, val_index in kf:
        training_data, validation_data = train.loc[train_index], train.loc[val_index]
        dtr = xgb.DMatrix(training_data[feat], label=np.log(training_data[y]+shift))
        dval = xgb.DMatrix(validation_data[feat], label=np.log(validation_data[y]+shift))
        watchlist = [(dtr, 'train'), (dval, 'eval')]
        history = dict()
        model0 = xgb.train(params, dtr,num_boost_round= 800, evals=watchlist,evals_result=history,  feval=evalerror,verbose_eval=50)
        pred =  model0.predict(dval)
        pred = np.exp(pred)-shift
        er.append(mean_absolute_error(pred,validation_data[y]))
        print("mae",mean_absolute_error(pred,validation_data[y]))
        train['pred'+method+str(i)].loc[val_index] = pred
        dtest = xgb.DMatrix(test[feat])
        proba = proba+(np.exp(model0.predict(dtest))-shift)
        imps = pd.Series(model0.get_fscore()).sort_values()
        h.append(imps)
        print('NEXT FOLD')
    test['pred'+method+str(i)] = proba*(1/fold)

In [ ]:

for i in range(2):
    feat = []
    rand = list(set(np.random.randint(0,len(feat_all),size=(len(feat_all)))))
    print(len(rand)/len(feat_all))
    for j in range(len(rand)):
        feat.append(feat_all[rand[j]])
    get_prob_xgb(train,test,feat,3,'loss','regr',i)
feat_prob = list(set(train.columns)-set(feat_all))
tr = train[feat_prob]
te = test[feat_prob]
te.head()

In [ ]:
shift = 200
def get_prob_lin(train,test,feat,fold,y,method,i):
    model0 = linear_model.LinearRegression()
    train['pred'+method+str(i)]=0
    test['pred'+method+str(i)] =0 
    er = []
    proba = 0
    kf = KFold(train.shape[0], n_folds=fold,shuffle=True)
    for train_index, val_index in kf:
        training_data, validation_data = train.loc[train_index], train.loc[val_index]
        model0.fit(np.log(training_data[feat]+shift),np.log(training_data[y]+shift))
        pred = model0.predict(np.log(validation_data[feat]+shift))
        pred = np.exp(pred)-shift
        er.append(mean_absolute_error(pred,validation_data[y]))
        print("mae",mean_absolute_error(pred,validation_data[y]))
        train['pred'+method+str(i)].loc[val_index] = pred
        proba = proba+(np.exp(model0.predict(np.log(test[feat]+shift)))-shift)
    print("mean",np.mean(er))
    test['pred'+method+str(i)]=(proba)*(1/fold)

In [ ]:
feat_all = list(set(train.columns) -{'id','loss'})
feat_all

In [ ]:

for i in range(10):
    feat = []
    rand = list(set(np.random.randint(0,len(feat_all),size=(len(feat_all)))))
    print(len(rand)/len(feat_all))
    for j in range(len(rand)):
        feat.append(feat_all[rand[j]])
    get_prob_lin(train,test,feat,3,'loss','regr',i)
feat_prob = list(set(train.columns)-set(feat_all))
tr = train[feat_prob]
te = test[feat_prob]
te.head()

In [ ]:
random.uniform(0, 20)
random.randint(1,27)

In [ ]:
shift=200
def get_prob_xgb(train,test,feat,fold,y,method,i):
    h = []
    seed = random.randint(1,1337)
    mcw = random.randint(0,10)
    eta = random.uniform(0.01, 0.05)
    cb = random.uniform(0.5, 1)
    md = random.randint(4,15)
    s = random.uniform(0.8, 1)
    er = []
    proba = 0
    params = {
            'min_child_weight': mcw,
            'eta': eta,
            'colsample_bytree': cb,
            'max_depth': md,
            'subsample': s,
            'alpha': 1,
            'gamma': 1,
            'silent': 1,
            'verbose_eval': True,
            'seed': seed
        }
    train['pred'+method+str(i)+str(mcw)+str(eta)+str(cb)+str(md)+str(s)]=0
    test['pred'+method+str(i)+str(mcw)+str(eta)+str(cb)+str(md)+str(s)] =0 
    kf = KFold(train.shape[0], n_folds=fold,shuffle=True)
    for train_index, val_index in kf:
        training_data, validation_data = train.loc[train_index], train.loc[val_index]
        dtr = xgb.DMatrix(training_data[feat], label=np.log(training_data[y]+shift))
        dval = xgb.DMatrix(validation_data[feat], label=np.log(validation_data[y]+shift))
        watchlist = [(dtr, 'train'), (dval, 'eval')]
        history = dict()
        model0 = xgb.train(params, dtr,num_boost_round= 1000, evals=watchlist,evals_result=history,  feval=evalerror,verbose_eval=50)
        pred =  model0.predict(dval)
        pred = np.exp(pred)-shift
        er.append(mean_absolute_error(pred,validation_data[y]))
        print("mae",mean_absolute_error(pred,validation_data[y]))
        train['pred'+method+str(i)+str(mcw)+str(eta)+str(cb)+str(md)+str(s)].loc[val_index] = pred
        dtest = xgb.DMatrix(test[feat])
        proba = proba+(np.exp(model0.predict(dtest))-shift)
        imps = pd.Series(model0.get_fscore()).sort_values()
        h.append(imps)
        print('NEXT FOLD')
    test['pred'+method+str(i)+str(mcw)+str(eta)+str(cb)+str(md)+str(s)] = proba*(1/fold)

In [ ]:
from numpy import random

In [ ]:
feat_all

In [ ]:

for i in range(2):
    feat = []
    rand = list(set(np.random.randint(0,len(feat_all),size=(len(feat_all)))))
    print(len(rand)/len(feat_all))
    for j in range(len(rand)):
        feat.append(feat_all[rand[j]])
    get_prob_xgb(train,test,feat_all,3,'loss','regr',i)
feat_prob = list(set(train.columns)-set(feat_all))
tr = train[feat_prob]
te = test[feat_prob]
te.head()

In [ ]:
tr.head()

In [ ]:
z = (tr.loss-tr["predregr040.0187086573326524370.9361401852393766100.9245917889729721"])>0
k=0
for i in range(len(z)):
    if z[i]==True:
        k=k+1

In [ ]:
min(tr["predregr040.0187086573326524370.9361401852393766100.9245917889729721"])

In [ ]:
min(tr.loss)

In [ ]:
mean_absolute_error((tr["predregr040.0187086573326524370.9361401852393766100.9245917889729721"]-75),tr.loss)

In [ ]:
Xtr, Xval, ytr, yval = train_test_split(tr,tr.loss,random_state = 7, test_size=0.2)
shift = 200
feat = list(set(te.columns)-{"id","loss"} )
dtr = xgb.DMatrix(Xtr[feat], label=np.log(ytr+shift))
dval = xgb.DMatrix(Xval[feat], label=np.log(yval+shift))
watchlist = [(dtr, 'train'), (dval, 'eval')]
history = dict()

In [ ]:
train['dif'] = train['predregr1'] - train['predregr0']

In [ ]:
train.head()

In [ ]:
mean_absolute_error((train['predregr060.0167500983541034420.887604619346874140.8687493386495285'])*(2/2),tr.loss)

In [ ]:
model = xgb.train(params, dtr,num_boost_round= int(2012 / 0.9), evals=watchlist,evals_result=history,  feval=evalerror,verbose_eval=50)

In [ ]:
predicti = np.exp(model.predict(dtest)) - shift

result:
non_feat_linear_non_param = 1219
all_feat = 1134

# Kaggle new metric

In [ ]:
def logregobj(preds, dtrain):
    labels = dtrain.get_label()
    con = 2
    x =preds-labels
    grad =con*x / (np.abs(x)+con)
    hess =con**2 / (np.abs(x)+con)**2
    return grad, hess 

def evalerror(preds, dtrain):
    labels = dtrain.get_label()
    return 'mae', mean_absolute_error(np.exp(preds), np.exp(labels))

In [ ]:
model = xgb.train(params, dtr,num_boost_round= int(2012 / 0.9) , evals=watchlist,evals_result=history, obj=logregobj, feval=evalerror, early_stopping_rounds=50,verbose_eval=50) 

In [ ]:
gam fo cont(sglazhivanie)
to_remove = function (x, prop = 0.99) { # prop is the proportion with default 99% 
factor_vars = sapply(x, class) == 'factor' 
remove_vars = numeric() 
for (i in 1:ncol(x)){ 
if (factor_vars[i] && any(table(x[, i])/nrow(x) >= prop)) remove_vars = c(remove_vars, i) 
} 
print(remove_vars) 
} 

In [ ]:
model = xgb.train(params, xgtrain,num_boost_round= int(2012 / 0.9) , evals=watchlist,evals_result=history, obj=logregobj, feval=evalerror, early_stopping_rounds=50,verbose_eval=50) 

predicti = np.exp(model.predict(xgtest)) - shift

# IMP FEATURES

In [ ]:
imps = pd.Series(model.get_fscore()).sort_values()

In [ ]:

imps.plot(kind='bar', figsize=(15,6),logy=False)
plt.show()

In [ ]:
imps.to_csv('dataf/id.csv')

In [ ]:
te.head()

In [ ]:
tr.head()

# Submission

In [ ]:
sub = pd.read_csv("data/sample_submission.csv")

In [ ]:
sub.head()

In [ ]:
sub['loss']=predicti

In [ ]:
sub.head()

In [ ]:
sub.to_csv('pred/xgb_mean5fold(cont14+id)(new12,2200,count).csv',index=False)

In [ ]:
pl=sub.loss

# Blending

In [ ]:
x1 = pd.read_csv("pred/xgb_kagglfeatboxcox(12,2200,count).csv")
x2 = pd.read_csv("pred/xgb_kaggl1(12,2200,3foldmean).csv")
x3 = pd.read_csv("pred/xgb_kaggl1(12,2200,getnan).csv")

In [ ]:
stats.pearsonr(x3.loss,x1.loss),stats.spearmanr(x3.loss,x1.loss)

In [ ]:
import scipy.stats as stats

In [ ]:
x1.head()

In [ ]:
x = x1.copy()

In [ ]:
x['loss'] = (x1['loss'] + x2['loss'] +x3["loss"])*(1/3)
x.head()

In [ ]:
sub.to_csv('pred/mean(kagglfeatboxcox(12,2200,count),xgb_kaggl1(12,2200,3foldmean)xgb_kaggl1(12,2200,getnan)).csv',index=False)